In [1]:
import os, sys
from sys import stdout
sys.path.append('..')
from utils.pdf_helper import extract_chapters

# 读取目录

## 读取单个目录

In [2]:
path = r'data/000792_盐湖股份/000792_盐湖股份_2023年年度报告_1219471218.pdf'
chapters = extract_chapters(path)

# 匹配管理层经营分析

## 最终方案

In [3]:
from utils.pdf_helper import read_management_chapter
page_content = read_management_chapter(path)

with open("test.md", "w", encoding="utf-8") as file:
    file.write(page_content)

## 步骤拆分
### 获取"管理层讨论与分析"起止页

In [4]:
from utils.pdf_helper import chapter_range

keywords = ["管理层讨论与分析", "董事会报告"]
for keyword in keywords:
    try:
        start_page, end_page, start_title, end_title = chapter_range(chapters, keyword)
        break
    except:
        continue
    
start_page, end_page, start_title, end_title

(10, 36, '第三节 管理层讨论与分析', '第四节 公司治理')

## 抽取章节内容
### PPStructure + pdfplumber
优点:可以OCR
缺点:速度比纯pdfplumber的慢x4

In [5]:
import fitz
from utils.pdf_helper import crop_page, recongize_layout, debug_layout, chapter_range, extract_content

page_content = ""
with fitz.open(path) as pdf:
    for page_num in range(start_page, end_page + 1): # range的特点,不+1的话会少一页
        page = pdf[page_num - 1] # 下标从0开始,所以要减1
        image = crop_page(page)
        page_layout = recongize_layout(image)
        if page_num == 11:
            # image.save("page_13.png")
            debug_layout(image, page_layout)
        page_content += extract_content(page, page_layout)


### pdfplumber实现

In [6]:
import re
import fitz
import pdfplumber
from utils.pdf_helper import find_header_and_footer, extract_content, debug_page

page_content = ""
hnf, page_size = find_header_and_footer(path, start_page, end_page) # header and footer
with pdfplumber.open(path) as pdf:
    for page_num in range(start_page, end_page + 1): # range的特点,不+1的话会少一页
        page = pdf.pages[page_num - 1]
        tables = page.find_tables()
        # 是否使用hnf模板
        page_layout = hnf[:] if (int(page.width), int(page.height)) == page_size else []
        
        if tables:
            for table in tables:
                bbox = table.bbox
                bbox = [bbox[0] - 1, bbox[1] - 1, bbox[2] + 1, bbox[3] + 1] # 扩大才能识别全
                page_layout.append({'type': 'table', 'bbox': bbox, 'content': table.extract()})

        page_content += extract_content(page, page_layout)
        if page_num == 11:
            debug_page(path, page_num - 1, page_layout)
            print(page_layout)

    # with open("test.md", "w", encoding="utf-8") as file:
    #     file.write(page_content)

[{'type': 'table', 'bbox': [0, 0, 595, 807], 'content': [[None, ''], ['', '西青海藏盐天湖路工股业股份份有有限限公公司司\nTQINIGBHEAIT S ATLTI LAAKNEL INUD UCSTORY. ，COL.,LTTDD'], [None, ''], ['', '第三节 管理层讨论与分析'], [None, '10']]}]
